In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px

In [ ]:
targetDataFile = 'coinone_btc_krw_hour_gmt.csv'

In [ ]:
dt_parser = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

hist_hour = pd.read_csv(
    targetDataFile, 
    index_col = 0, 
    parse_dates = True, 
    date_parser = dt_parser)

# UTC + 9로 시간 변환
# hist_hour.index = hist_hour.index + datetime.timedelta(hours=9)    

In [ ]:
def do_ampm(hist_hour, buy, hold):
    # 하....DataFrame.between_time에 이미 있다 -ㅅ-
    buyTime = datetime.time(buy, 0, 0)
    holdingTime = hold

    buyOffset = hist_hour.index.hour - buyTime.hour
    hist_hour['buyOffset'] = np.where(buyOffset >= 0, buyOffset, buyOffset + 24)
    hist_hour_include = hist_hour[ hist_hour['buyOffset'] < holdingTime ]
    hist_hour_exclude = hist_hour[ hist_hour['buyOffset'] >= holdingTime ]

    offset = datetime.timedelta(hours=buyTime.hour, minutes=buyTime.minute, seconds=0)
    buySheet = hist_hour_include.resample('1D', offset=offset).agg( {
        'open': 'first', #lambda df: None if df.empty else df[0],
        'close': 'last', #lambda df: None if df.empty else df[-1],
        'volume': 'sum',
        'value': 'sum',
    })
    excludeSheet = hist_hour_exclude.resample('1D', offset=offset).agg( {
        'open': 'first', #lambda df: None if df.empty else df[0],
        'close': 'last', #lambda df: None if df.empty else df[-1],
        'volume': 'sum',
        'value': 'sum',
    })

    # drop first 2 days
    # head data cannot be reliable
    buySheet.drop(buySheet.index[:2], inplace=True)

    buySheet.index.name = 'time_of_buy'
    buySheet['simple.return_of_rate'] = buySheet['open'].shift(-1) / buySheet['open']
    buySheet['simple.assets_after_sell'] = buySheet['simple.return_of_rate'].cumprod()
    buySheet['simple.assets_max'] = buySheet['simple.assets_after_sell'].cummax()
    buySheet['simple.dd'] = 1 - buySheet['simple.assets_after_sell'] / buySheet['simple.assets_max']
    buySheet['simple.mdd'] = buySheet['simple.dd'].cummax()

    buySheet['ampm.return_of_rate'] = buySheet['close'] / buySheet['open']
    buySheet['ampm.assets_after_sell'] = buySheet['ampm.return_of_rate'].cumprod()
    buySheet['ampm.assets_max'] = buySheet['ampm.assets_after_sell'].cummax()
    buySheet['ampm.dd'] = 1 - buySheet['ampm.assets_after_sell'] / buySheet['ampm.assets_max']
    buySheet['ampm.mdd'] = buySheet['ampm.dd'].cummax()

    buySheet['ampm.volumeDiff'] = excludeSheet['volume'] - buySheet['volume']
    buySheet['ampm.volumeDiffSum'] = buySheet['ampm.volumeDiff'].cumsum()

    return buySheet

In [ ]:
maxAsset = 0

results_bytime = pd.DataFrame(index=range(0, 24), columns=range(1,24), dtype=float)
results_byassets = pd.DataFrame(columns=["assets", "buyTime", "holdingTime", "mdd", "volumeDiffSum"])
for i in range(0, 24):
    for j in range(1, 24):
      buySheet = do_ampm(hist_hour, i, j)
      if maxAsset < buySheet['ampm.assets_after_sell'][-2]:
        maxAsset = buySheet['ampm.assets_after_sell'][-2]
        maxBt = i
        maxHt = j
        maxBuySheet = buySheet
      results_bytime.iloc[i][j] = buySheet['ampm.assets_after_sell'][-2]
      results_byassets = results_byassets.append(
        {
          "assets": buySheet['ampm.assets_after_sell'][-2], 
          "buyTime": i, 
          "holdingTime": j, 
          "mdd": buySheet['ampm.mdd'][-2], 
          "volumeDiffSum": buySheet['ampm.volumeDiffSum'][-2],
        },
        ignore_index=True,
      )


In [ ]:
results_bytime.index.name = "Buy Time"
results_byassets.set_index("assets", inplace=True)
results_byassets = results_byassets.sort_index(ascending=False)

In [ ]:
results_bytime_data = results_bytime.to_csv(float_format="%.02f")
results_byFinalAsset_data = results_byassets.to_csv(float_format="%.02f")

with open(targetDataFile+"_results_ampm.csv", "w", encoding='utf-8', newline='') as f:
  f.write(results_bytime_data)
  f.write("\r\n")
  f.write(results_byFinalAsset_data)
  f.write("\r\n")

print(maxBuySheet['simple.mdd'])
px.line(maxBuySheet, y=['simple.assets_after_sell', 'ampm.assets_after_sell'], log_y=True).show()
px.line(maxBuySheet, y=['ampm.volumeDiffSum']).show()